In [1]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=/kaggle/input/pytorch-geometric-packages

Looking in links: /kaggle/input/pytorch-geometric-packages
Processing /kaggle/input/pytorch-geometric-packages/torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/pytorch-geometric-packages/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/pytorch-geometric-packages/torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/pytorch-geometric-packages/torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl
Processing /kaggle/input/pytorch-geometric-packages/torch_geometric-2.5.3-py3-none-any.whl


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, GCNConv, LayerNorm, GraphNorm, InstanceNorm
from torch_geometric.nn.models import GAT, GraphSAGE, PMLP
from torch_geometric.nn.pool import max_pool
from torch_geometric.nn.norm import BatchNorm
from torch_geometric.utils import to_dense_adj
from torch_geometric.utils import dropout_edge
from scipy.spatial import distance_matrix
import networkx as nx
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold
from torchvision.models.feature_extraction import create_feature_extractor
from torch_geometric.nn.pool import global_max_pool

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
import random

## Config

In [3]:
batch_size = 10
num_epochs = 50
kfold = 5
alpha = 0.25
gamma = 2
beta = 0.1

## Modeling

### Prepare data

In [4]:
df = pd.read_csv('/kaggle/input/isic-2024-challenge/train-metadata.csv')

In [5]:
num_positive = df[df.target == 1].shape[0]
num_negative = df[df.target == 0].shape[0]
total = num_positive + num_negative
class_weight = torch.tensor([total/num_negative, total/num_positive]).cuda()

In [6]:
#df_eff = pd.read_csv("/kaggle/input/skin-cancer-image-model-output/output_eff.csv")
#df_eff = pd.concat([df.copy(), df_eff], axis=1)
#df_eff = df_eff[['patient_id', "target_effnetv1b0"]]
#df_eva = pd.read_csv("/kaggle/input/skin-cancer-image-model-output/output_eva.csv")
#df_eva = pd.concat([df.copy(), df_eva], axis=1)
#df_eva = df_eva[['patient_id', "target_eva02"]]

In [7]:
data = []
label_img = []
label_node = []
data_size = []
for id_ in tqdm(df.patient_id.unique()):
    data_i = df[df.patient_id == id_]
    label_node.append(np.array(data_i.target))
    label_img.append(data_i.target.max())
    data.append(data_i)
    data_size.append(data_i.shape[0])

100%|██████████| 1042/1042 [01:11<00:00, 14.54it/s]


In [8]:
data_list = []
for i, data_ in tqdm(enumerate(data)):
    id_ = data_.patient_id.iloc[0]
    node_label = torch.tensor(np.array(data_.target))
    #node_label = torch.tensor(np.array(data_.target.max()))
    #node_feature_eff = np.array(df_eff[df.patient_id == id_]['target_effnetv1b0'])[:, None].astype('float32')
    #node_feature_eva = np.array(df_eva[df.patient_id == id_]['target_eva02'])[:, None].astype('float32')
    #node_feature_meta = torch.tensor(np.load(f'/kaggle/input/skin-cancer-graph-efficientnet/node_features/{id_}_node_feature_meta.npy').astype('float32'))
    #node_feature = torch.tensor(np.concatenate([node_feature_eff, node_feature_eva], axis=-1))
    node_feature = torch.tensor(np.load(f'/kaggle/input/skin-cancer-graph-efficientnet/node_features/{id_}_node_feature_img.npy').astype('float32'))
    edge_index = torch.tensor(np.load(f'/kaggle/input/img-patient-wise-structural-relation-of-skin-c/pos/{id_}_edge_index.npy'))
    pos = torch.tensor(np.load(f'/kaggle/input/img-patient-wise-structural-relation-of-skin-c/edge_index/{id_}_pos.npy').astype('float32'))
    graph = Data(x=node_feature, y = node_label, pos=pos, edge_index=edge_index)
    data_list.append(graph)

1042it [00:16, 62.41it/s]


### Build Model

In [9]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='sum'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        CE_loss = F.cross_entropy(inputs, targets, weight = class_weight, reduction='none')
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * (1 - pt) ** self.gamma * CE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()/batch_size
        else:
            return F_loss
        
        
def graph_laplacian_loss(node_embeddings, batch):
    edge_index = batch.edge_index.cuda()
    batch = batch.batch.cuda()
    if edge_index.shape[0]==0:
        return torch.tensor(0.)
    adjacency_matrix = to_dense_adj(edge_index, batch=batch)
    degree_matrix = torch.diag(adjacency_matrix.sum(dim=1))
    laplacian_matrix = degree_matrix - adjacency_matrix
    smoothness_loss = torch.trace(torch.matmul(node_embeddings.T, torch.matmul(laplacian_matrix, node_embeddings)))
    return smoothness_loss


def augmentation(edge_index, p=0.4):
    edge_index, edge_attr = dropout_edge(edge_index, p=p)
    return edge_index


def entropy_loss(logits):
    p = logits.softmax(dim=-1)
    loss = torch.sum(- p *p.log())/batch_size
    return loss

In [10]:
class GNN_Model(nn.Module):
    def __init__(self, in_channels = 128, hidden_channels = 128, out_channels = 2):
        super().__init__()
        self.in_channels = in_channels
        self.graphsage = GraphSAGE(in_channels, hidden_channels,
                    6, out_channels, jk='lstm', norm = LayerNorm(hidden_channels),
                    dropout = 0.3,  normalize=True)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        if edge_index.shape[0]==0:
            edge_index = torch.tensor([[0, 0]]).T.cuda()
        x = x.cuda()
        edge_index = edge_index.cuda()
        batch = batch.cuda()
        logits = self.graphsage(x, edge_index, batch=batch)
        return logits
        
    
    def predict(self, batch):
        with torch.no_grad():
            logits = self.forward(batch)
            pred = logits.softmax(dim=-1)[:, 1]
        return pred
    
    def fit(self, train_loader, optimizer, fold):
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
        best_score = 0.
        count=0
        for epoch in range(num_epochs):
            loop = tqdm(train_loader)
            self.train()
            for batch in loop:
                y = batch.y.cuda()
                logits = self.forward(batch)
                l_cls = FocalLoss(alpha = alpha, gamma=gamma)(logits, y)
                l_ent = entropy_loss(logits)
                loss = l_cls + l_ent
                #loss = nn.CrossEntropyLoss(weight=class_weight)(logits, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                torch.cuda.empty_cache()
                loop.set_postfix(loss=f'epoch: {epoch}--loss: {loss} --l_cls: {l_cls} -- l_ent: {l_ent}')
            scheduler.step()
            if epoch%1==0:
                self.eval()
                labels = []
                preds = []
                for batch in valid_loader:
                    label = batch.y
                    pred = self.predict(batch)
                    preds.append(pred)
                    labels.append(label)
                    torch.cuda.empty_cache()
                preds = torch.cat(preds).cpu().numpy()
                labels = torch.cat(labels).cpu().numpy()
                score = comp_score(pd.DataFrame(labels, columns = ['target']),
                                   pd.DataFrame(preds, columns=["prediction"]), "")
                
                if score>best_score:
                    count=0
                    best_score = score
                    torch.save(self.state_dict(), f'/kaggle/working/model_{fold}.pt')
                else:
                    count+=1
                    if count==6:
                        break
                print(f"epoch: {epoch} - Partial AUC Score: {score:.5f}")

### Kfold Validation

In [11]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [12]:
def comp_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80):
    v_gt = abs(np.asarray(solution.values)-1)
    v_pred = np.array([1.0 - x for x in submission.values])
    max_fpr = abs(1-min_tpr)
    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
    # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    return partial_auc

In [13]:
set_seed()

In [14]:
df_train = df.copy()

In [15]:
gkf = GroupKFold(n_splits=kfold)
df_train["fold"] = -1
for idx, (train_idx, val_idx) in enumerate(gkf.split(df_train, df_train["target"], groups=df_train["patient_id"])):
    df_train.loc[val_idx, "fold"] = idx

In [16]:
models = []
for fold in range(5):
    _df_train = df_train[df_train["fold"] != fold].reset_index(drop=True)
    _df_valid = df_train[df_train["fold"] == fold].reset_index(drop=True)
    #train
    mask = np.isin(df.patient_id.unique(), _df_train.patient_id.unique())
    idx = np.where(mask)[0]
    data_train_list = list(map(lambda i: data_list[i] ,idx))
    #val
    mask = np.isin(df.patient_id.unique(), _df_valid.patient_id.unique())
    idx = np.where(mask)[0]
    data_valid_list = list(map(lambda i: data_list[i] ,idx))

    #loader
    train_loader = DataLoader(data_train_list, batch_size=batch_size, shuffle=True, 
                              num_workers = 4, persistent_workers=True)
    valid_loader = DataLoader(data_valid_list, batch_size=batch_size, shuffle=False,
                              num_workers = 4, persistent_workers=True)
    
    #modeling
    model = GNN_Model().cuda()
    optimizer = optim.AdamW(model.parameters())
    model.fit(train_loader, optimizer, fold)
    models.append(model)
    torch.cuda.empty_cache()

100%|██████████| 84/84 [00:13<00:00,  6.24it/s, loss=epoch: 0--loss: 39.478397369384766 --l_cls: 3.4125454425811768 -- l_ent: 36.065853118896484]


epoch: 0 - Partial AUC Score: 0.10634


100%|██████████| 84/84 [00:13<00:00,  6.44it/s, loss=epoch: 1--loss: 23.951841354370117 --l_cls: 3.0293314456939697 -- l_ent: 20.922510147094727]


epoch: 1 - Partial AUC Score: 0.11973


100%|██████████| 84/84 [00:12<00:00,  6.67it/s, loss=epoch: 2--loss: 120.65739440917969 --l_cls: 95.93675994873047 -- l_ent: 24.720630645751953]


...

100%|██████████| 84/84 [00:13<00:00,  6.06it/s, loss=epoch: 7--loss: 1.781416654586792 --l_cls: 0.7897208333015442 -- l_ent: 0.9916958212852478]


epoch: 7 - Partial AUC Score: 0.07510


100%|██████████| 84/84 [00:14<00:00,  5.99it/s, loss=epoch: 8--loss: 7.570220947265625 --l_cls: 3.073209047317505 -- l_ent: 4.497011661529541]


epoch: 8 - Partial AUC Score: 0.08106


100%|██████████| 84/84 [00:13<00:00,  6.00it/s, loss=epoch: 9--loss: 18.415102005004883 --l_cls: 7.700491428375244 -- l_ent: 10.714611053466797]


In [17]:
loader = DataLoader(data_list, batch_size=batch_size, shuffle=False, 
                              num_workers = 4, persistent_workers=True)

In [18]:
patient_ids = df.patient_id.unique()
df_preds = []

for fold in range(5):
    model = models[fold]
    model.load_state_dict(torch.load(f'/kaggle/working/model_{fold}.pt'))
    model.eval()
    df_pred = pd.DataFrame(df[['isic_id', 'patient_id']], columns=['isic_id', 'patient_id', f'pred_{fold}'],
                               index=df.index)
    for i, batch in enumerate(tqdm(loader)):
        preds = model.predict(batch)
        batch_idx = batch.batch.unique()
        for idx in batch_idx:
            pred_i = preds[batch.batch == idx].cpu().numpy()
            sample_idx = idx + i * batch_size
            patient_id = patient_ids[sample_idx]
            df_pred.loc[df_pred.patient_id == patient_id, f'pred_{fold}'] = pred_i
    df_preds.append(df_pred)
df_preds = pd.concat(df_preds, axis=1)

100%|██████████| 105/105 [01:18<00:00,  1.34it/s]


In [19]:
df_preds.to_csv('/kaggle/working/gnn_preds.csv')

In [20]:
#df_preds[df.target==1].hist()